importing modules and files

In [1]:
import xml.etree.ElementTree as ET 
from Api.apiRequests import getJobListings
from operatingSys.servieces import importXml

Importing and providing the xml data from server to local, then parsing to be usable

In [2]:
url = 'https://kunden.jobadpartner.de/publisher/api/jobvector_2020/?accessCode=WfxGIuBy' #source XML address
xmlData = getJobListings(url)

#writes the fetched xml to desired address
xmlFolderAddress = "xmlFiles/fetchedXml.xml" # the target folder that we have imported our xml response

importXml(xmlFolderAddress,xmlData)

root = ET.parse(xmlFolderAddress).getroot()

In [3]:
for joblisting in root:
    for i in (joblisting.iter('language')):
        if i.text == 'EN': #this will be changed to not equal to in later commits
            for elem in joblisting.iter('introduction'):
                print(elem.text)



            <p>Datics Consulting is a growing technology service startup in Cloud, DevOps, IT-Outsourcing, Data Engineering, and AI.</p><p><br/>&amp;amp;nbsp;</p><p>Cloud and Devops themes are your passion?<br/>Datics Consulting is searching for M&amp;amp;uuml;nchen, Frankfurt, D&amp;amp;uuml;sseldorf, Berlin or Remote -</p><p>Consultants for Cloud Engineering (m/f/d)</p> 

        
